In [1]:
import os  
import sys 
import uuid
import activelearning as al
from activelearning.config import cfg, Directories
import time
import fnmatch

sys.path.insert(0, os.path.abspath('..')) 
default_cluster_id = cfg.PARALLEL.CLUSTER_ID

2023-06-20 20:39:59.480456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def run_job(notebook_path,
            config_path, 
            name="active_learning", 
            partition="gpu", 
            n_cpus=6, ram="64GB",
            max_time="120:00:00", ):
    s = ""

    s += f" --job-name={name}"
    s += f" --partition={partition}"
    s += f" --time={max_time}"

    s += f" --nodes=1"
    s += f" --cpus-per-task={n_cpus}"
    
    if partition in ("gpu", "dgx"):
        s += " --gres=gpu:1"
        
    s += f" --mem={ram}"

    s += " --output=logs/tjob_%A_%a_out.txt"
    s += " --error=logs/tjob_%A_%a_err.txt"

    s += f" -D {os.getcwd()}"

    # -- Beginning of the sh script to run the job
    s += " --wrap=\""
    #
    s += "echo 'jobid $SLURM_JOB_ID';"

    s += "python run.py"
    s += f" -config_path {config_path}"
    s += f" -notebook_path {notebook_path}"
    #

    s += "\""
    # -- End of sh script

    print(s)
    os.system(f"sbatch {s}")


Please note that any change un the configuration parameters here will not be propagated (because the config file just gets copied).

In [3]:
cfg.merge_from_file("configs/cavities/cavities.yaml")

config_path = "configs/cavities/six-random-small-noise-couplingrandom.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] +"_2023_again_2"

config_path = "configs/cavities/six-random-small-noise-paper.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] + "_june"

In [3]:
config_path = "configs/qubits.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] #+ "_bugfix_retry"

In [3]:
config_path = "configs/qubits-4-nometric.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0]

In [3]:
config_path = "configs/qubits-binomial.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] +"_retry"

In [5]:
config_path = "configs/qubits-binomial-4.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] + "_june3_metric"

In [3]:
config_path = "configs/qubits-binomial-4-smallcount.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] + "_june2_metric"

In [3]:
config_path = "configs/qubits-binomial-4-nometric-longer.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] + "_retry"

In [3]:
config_path = "configs/qubits-4-nometric.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] +"_june"

In [3]:
config_path = "configs/qubits-binary-4-nometric.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] +"_june_latestopping"

In [3]:
config_path = "configs/qubits-binomial-4-nometric.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] +"_june_latestopping"

In [9]:
config_path = "configs/qubits-many-measurements.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] + "_attempt2"

In [3]:
config_path = "configs/linear-toy-2023-changestest.yaml"
cfg.PATH.EXPERIMENT_NAME = config_path.split("/")[-1].split(".")[0] + "_old_again" #"-2023_larger_batchsize"
# cfg.PATH.EXPERIMENT_NAME = "test_linear"

In [5]:
notebook_path =  "notebook.ipynb"
partition="dgx"
# partition="standard"
# partition="highfreq"

cfg.ADVISOR.NAME = "InformationGainAdvisor"
# cfg.ADVISOR.NAME = "RandomAdvisor"
# cfg.ADVISOR.NAME = "FixedAdvisor"
cfg.ADVISOR.NAME = "UniformGridAdvisor"

cfg.merge_from_file(config_path)
cfg.PATH.SUBLABEL =  cfg.ADVISOR.NAME

Restore job

In [12]:
if os.path.exists(al.config.Directories().PATH_OUTPUT_EXPERIMENT):
    print("Resuming checkpoint")
    for file in fnmatch.filter(os.listdir(al.config.Directories().PATH_OUTPUT_CONFIGS), 
                               f'{cfg.PATH.SUBLABEL}*'):
        cfg.PATH.SUBLABEL =  file.split(".yaml")[0]
        load_path = f"{al.config.Directories().PATH_OUTPUT_CONFIGS}/{file}"
        cfg.merge_from_file(load_path)
        
        print(cfg.PATH.SUBLABEL)
        # print(notebook_path, load_path,partition)
        
        if cfg.PATH.SUBLABEL == "InformationGainAdvisor_180efce9-13df-4926-b424-1e76fed08337":
                run_job(notebook_path, 
                        config_path=load_path,
                        partition=partition)
                
                if cfg.ADVISOR.NAME =="InformationGainAdvisor": 
                        time.sleep(70) # give time to load
else:
        print("Experiment does not exist yet. Cannot resume, create a new experiment.")

Resuming checkpoint
InformationGainAdvisor_646a46e9-8b99-4930-92df-2d0ef3fc17f3
InformationGainAdvisor_180efce9-13df-4926-b424-1e76fed08337
 --job-name=active_learning --partition=gpu --time=120:00:00 --nodes=1 --cpus-per-task=6 --gres=gpu:1 --mem=64GB --output=logs/tjob_%A_%a_out.txt --error=logs/tjob_%A_%a_err.txt -D /zeropoint/u/lsarra/notebooks/active-learning-nf-test --wrap="echo 'jobid $SLURM_JOB_ID';python run.py -config_path outputs/qubits-binomial-4_june3_metric/configs/InformationGainAdvisor_180efce9-13df-4926-b424-1e76fed08337.yaml -notebook_path notebook.ipynb"
Submitted batch job 6137149
InformationGainAdvisor_5048fcfd-6e2a-4fa3-b0e9-ebf1bf537c16


New Run

In [6]:
# cfg.ADVISOR.N_TRAIN = 6000
# cfg.TRAINING.N_MEASUREMENTS = 400

n_jobs = 3

cfg.PARALLEL.N_NODES = 30
# cfg.LEARNER.POSTERIOR.TYPE = "Gaussian"
# cfg.PATH.EXPERIMENT_NAME = "barrier-10-small-noise-extended"

random_sublabel = True
cfg.PARALLEL.ENABLED = True if cfg.ADVISOR.NAME =="InformationGainAdvisor" else False

# "parallel_standard" (cpu) / "parallel_jpt" (gpu)
if partition=="standard":
    cfg.PARALLEL.PROFILE_ID = "parallel_standard"
else: 
    cfg.PARALLEL.PROFILE_ID = "parallel_jpt"

In [7]:
for _ in range(n_jobs):
    if random_sublabel:
        rnd_id = f"_{str(uuid.uuid4())}"
        cfg.PATH.SUBLABEL =  cfg.ADVISOR.NAME + rnd_id
        cfg.PARALLEL.CLUSTER_ID = default_cluster_id + rnd_id

    new_config_path = f"{al.config.Directories().PATH_OUTPUT_CONFIGS}/{cfg.PATH.SUBLABEL}.yaml"
    al.utils.init_output_directories()

    run_job(notebook_path, new_config_path,
            partition=partition)
    
        
    if cfg.ADVISOR.NAME =="InformationGainAdvisor": 
        time.sleep(70) # give time to launch parallel nodes

Creating directories...
 --job-name=active_learning --partition=dgx --time=120:00:00 --nodes=1 --cpus-per-task=6 --gres=gpu:1 --mem=64GB --output=logs/tjob_%A_%a_out.txt --error=logs/tjob_%A_%a_err.txt -D /zeropoint/u/lsarra/notebooks/active-learning-nf-test --wrap="echo 'jobid $SLURM_JOB_ID';python run.py -config_path outputs/six-random-small-noise-paper_june/configs/UniformGridAdvisor_e3e5fc26-1fe6-473f-a037-8059c0c32089.yaml -notebook_path notebook.ipynb"
Submitted batch job 6168184
Creating directories...
 --job-name=active_learning --partition=dgx --time=120:00:00 --nodes=1 --cpus-per-task=6 --gres=gpu:1 --mem=64GB --output=logs/tjob_%A_%a_out.txt --error=logs/tjob_%A_%a_err.txt -D /zeropoint/u/lsarra/notebooks/active-learning-nf-test --wrap="echo 'jobid $SLURM_JOB_ID';python run.py -config_path outputs/six-random-small-noise-paper_june/configs/UniformGridAdvisor_a1d45b11-8466-4b53-902b-abe09eba7c13.yaml -notebook_path notebook.ipynb"
Submitted batch job 6168185
Creating directori